<a href="https://colab.research.google.com/github/Rajani1-tech/-100daysofcode/blob/main/Fake_News_Classifier_Using_Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Classifier Using Lstm

In [ ]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/train.csv')

In [6]:
df.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [5]:
df.tail(3)

,id,title,author,text,label
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [7]:
df = df.dropna()

In [8]:
X = df.drop('label', axis=1)

In [9]:
y = df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf


In [16]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [17]:
voc_size= 5000

# Onehot Representation

In [25]:
messages = X.copy()

In [26]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [27]:
messages.reset_index(inplace=True)

In [28]:
import nltk
import re
from nltk.corpus import stopwords

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [31]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [32]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4073, 1286, 1448, 4158, 4309, 2661, 1491, 4697, 3586, 2627],
 [4578, 3896, 4617, 995, 3573, 3215, 381],
 [1002, 167, 51, 3455],
 [690, 2493, 3419, 2445, 4040, 243],
 [494, 3573, 328, 1132, 389, 266, 3573, 2530, 3588, 880],
 [2047,
  69,
  4205,
  589,
  4764,
  3932,
  1841,
  954,
  3614,
  373,
  1116,
  4329,
  4116,
  1663,
  381],
 [985, 3679, 4883, 4502, 2497, 1956, 3935, 3025, 120, 2611, 4381],
 [30, 1459, 677, 900, 4102, 645, 3932, 2357, 120, 2611, 4381],
 [1734, 123, 776, 1739, 2331, 2932, 1788, 241, 3932, 71],
 [1248, 4169, 427, 817, 4462, 366, 3796, 3999],
 [4439, 4466, 3780, 2118, 3745, 51, 259, 918, 1182, 765, 1863],
 [2445, 2417, 4309, 2932, 3932, 4102],
 [359, 4910, 3431, 3032, 1277, 1298, 4026, 3248, 736],
 [1105, 4851, 3885, 4323, 4427, 1044, 2630, 120, 2611, 4381],
 [79, 1574, 580, 4363, 2711, 120, 2611, 4381],
 [3077, 565, 2038, 4931, 3619, 24, 2178, 258, 2994, 3090],
 [590, 3764, 3896],
 [192, 2867, 2494, 461, 3932, 1949, 1454, 381],
 [3803, 1693, 4617, 4625, 289,

# Embedding Representation

In [33]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4697 3586 2627]
 [   0    0    0 ... 3573 3215  381]
 [   0    0    0 ...  167   51 3455]
 ...
 [   0    0    0 ...  120 2611 4381]
 [   0    0    0 ... 4521 4753 3252]
 [   0    0    0 ... 1164 4453 2670]]


In [34]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4073,
       1286, 1448, 4158, 4309, 2661, 1491, 4697, 3586, 2627], dtype=int32)

# Creating model

In [35]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
len(embedded_docs), y.shape

(18285, (18285,))

In [37]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [38]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


# Model Training

In [40]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 12s 42ms/step - loss: 0.3308 - accuracy: 0.8450 - val_loss: 0.1892 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 9s 45ms/step - loss: 0.1367 - accuracy: 0.9443 - val_loss: 0.1893 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0975 - accuracy: 0.9630 - val_loss: 0.3043 - val_accuracy: 0.9157
Epoch 4/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0728 - accuracy: 0.9735 - val_loss: 0.2378 - val_accuracy: 0.9087
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0469 - accuracy: 0.9842 - val_loss: 0.2590 - val_accuracy: 0.9123
Epoch 6/10
192/192 [==============================] - 12s 61ms/step - loss: 0.0283 - accuracy: 0.9925 - val_loss: 0.2951 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 14s 72ms/step - loss: 0.0165 - accuracy: 0.9946 - val_loss: 0.4015 - val_accuracy: 0.

# Adding Dropout

In [41]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

              